In [2]:
"""
TensorFlow Task
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()


dataset_path ="data/Iris.csv"
df = pd.read_csv(dataset_path)


df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


learning_rate = 0.01
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1


X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])


get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):

    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

                          
logits = example_net(X)


loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()



with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

/tmp/ipykernel_45258/3331397441.py:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = y.astype(np.int)[:, np.newaxis]
/tmp/ipykernel_45258/3331397441.py:52: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.

Epoch 0, loss : 12.3026, val_loss : 27.5008, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 1.6502, val_loss : 1.6921, acc : 0.750, val_acc : 0.688
Epoch 2, loss : 9.8394, val_loss : 6.5995, acc : 0.250, val_acc : 0.625
Epoch 3, loss : 0.0179, val_loss : 1.8780, acc : 1.000, val_acc : 0.688
Epoch 4, loss : 0.0000, val_loss : 1.2362, acc : 1.000, val_acc : 0.812
Epoch 5, loss : 0.0000, val_loss : 0.0908, acc : 1.000, val_acc : 0.938
Epoch 6, loss : 0.0008, val_loss : 3.0719, acc : 1.000, val_acc : 0.750
Epoch 7, loss : 0.0000, val_loss : 0.0342, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.0000, val_loss : 1.6674, acc : 1.000, val_acc : 0.812
Epoch 9, loss : 0.0000, val_loss : 0.6257, acc : 1.000, val_acc : 0.812
Epoch 10, loss : 0.0001, val_loss : 2.3368, acc : 1.000, val_acc : 0.750
Epoch 11, loss : 0.0000, val_loss : 0.0006, acc : 1.000, val_acc : 1.000
Epoch 12, loss : 0.0000, val_loss : 0.0092, acc : 1.000, val_acc : 1.000
Epoch 13, loss : 0.0001, val_loss : 2.3725, acc : 1.000, va

###### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Iris Species
サンプルコードと同じくこの中のtrain.csvを使用してください。目的変数はSpeciesに含まれる3種類全てを使います。
2クラスの分類と3クラス以上の分類の違いを考慮してください。
それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

In [3]:
from sklearn.preprocessing import OneHotEncoder

dataset_path ="data/Iris.csv"
df = pd.read_csv(dataset_path)

y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y[y=='Iris-setosa'] = 2

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:,np.newaxis])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

/home/lkhagvadorj/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
y_train.shape

(96, 3)

In [5]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [6]:
learning_rate = 0.01
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

   
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

                            
logits = example_net(X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()



with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

/tmp/ipykernel_45258/1704943362.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
/tmp/ipykernel_45258/2084279389.py:56: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional informa

Epoch 0, loss : 16.3418, val_loss : 38.3685, acc : 0.833, val_acc : 0.500
Epoch 1, loss : 0.0000, val_loss : 2.1807, acc : 1.000, val_acc : 0.833
Epoch 2, loss : 0.0000, val_loss : 2.2918, acc : 1.000, val_acc : 0.833
Epoch 3, loss : 0.0000, val_loss : 1.2339, acc : 1.000, val_acc : 0.875
Epoch 4, loss : 0.0000, val_loss : 2.7925, acc : 1.000, val_acc : 0.917
Epoch 5, loss : 0.0000, val_loss : 1.0276, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0000, val_loss : 0.8947, acc : 1.000, val_acc : 0.875
Epoch 7, loss : 0.0000, val_loss : 0.9794, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.0000, val_loss : 0.7442, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 1.0601, acc : 1.000, val_acc : 0.917
Epoch 10, loss : 0.0000, val_loss : 1.9115, acc : 1.000, val_acc : 0.917
Epoch 11, loss : 0.0000, val_loss : 1.8436, acc : 1.000, val_acc : 0.917
Epoch 12, loss : 0.0000, val_loss : 2.0354, acc : 1.000, val_acc : 0.917
Epoch 13, loss : 0.0000, val_loss : 1.0605, acc : 1.000, va

House Prices: Advanced Regression Techniques

In [7]:
import pandas as pd
import numpy as np
data_ori = pd.read_csv("data/train.csv")

In [8]:
max_num_of_row = len(data_ori)
data_loss_rate = data_ori.isnull().sum() / max_num_of_row * 100

In [9]:
data_drop_5nan_col = data_ori.dropna(axis=1, thresh=max_num_of_row-5)
print("before:{}".format(len(data_ori.columns.values)))
print("after:{}".format(len(data_drop_5nan_col.columns.values)))

before:81
after:63


In [10]:
data_to_use = data_drop_5nan_col.dropna(axis=0)
data_to_use = data_to_use.reset_index(drop=True)

In [11]:
col_featuer = ['OverallQual',
       'GrLivArea',
       'GarageCars',
       'GarageArea',
       'TotalBsmtSF',
       '1stFlrSF',
       'FullBath',
       'TotRmsAbvGrd',
       'YearBuilt',
       'YearRemodAdd',
      ]

col_target = ['SalePrice']

col = col_featuer + col_target

In [12]:
data = data_to_use[col]

In [13]:
X = data[col_featuer].values
Y = data[col_target].values

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [15]:
learning_rate = 0.01
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = x_train.shape[1]
n_samples = x_train.shape[0]
n_classes = 1

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(x_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

                            
logits = example_net(X)
loss_op = tf.reduce_mean(tf.abs(Y - logits))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
correct_pred = logits

accuracy = 1 - (tf.reduce_sum(tf.square(Y - correct_pred)) / tf.reduce_sum(tf.square(Y - tf.reduce_mean(Y))))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: x_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: x_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

/tmp/ipykernel_45258/1704943362.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
/tmp/ipykernel_45258/469766225.py:54: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional informat

Epoch 0, loss : 37774.9023, val_loss : 42120.8242, acc : 0.909, val_acc : 0.328
Epoch 1, loss : 34963.9375, val_loss : 36535.3633, acc : 0.931, val_acc : 0.417
Epoch 2, loss : 25081.0469, val_loss : 39177.1562, acc : 0.964, val_acc : 0.334
Epoch 3, loss : 24401.3223, val_loss : 38400.3242, acc : 0.968, val_acc : 0.344
Epoch 4, loss : 22653.6934, val_loss : 37356.7656, acc : 0.972, val_acc : 0.337
Epoch 5, loss : 21786.9590, val_loss : 35426.8477, acc : 0.977, val_acc : 0.355
Epoch 6, loss : 29473.1777, val_loss : 33439.9062, acc : 0.958, val_acc : 0.429
Epoch 7, loss : 25761.4062, val_loss : 35153.5000, acc : 0.968, val_acc : 0.348
Epoch 8, loss : 18827.7559, val_loss : 37107.3438, acc : 0.979, val_acc : 0.304
Epoch 9, loss : 21917.6562, val_loss : 33556.1016, acc : 0.976, val_acc : 0.353
Epoch 10, loss : 19800.0996, val_loss : 32508.3672, acc : 0.978, val_acc : 0.389
Epoch 11, loss : 20359.9746, val_loss : 34103.6562, acc : 0.978, val_acc : 0.375
Epoch 12, loss : 24589.2656, val_loss 

###### 【Task5】MNIST Dataset

In [16]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype(np.float)
x_test = x_test.astype(np.float)
x_train /= 255
x_test /= 255
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

/tmp/ipykernel_45258/1522112106.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_train = x_train.astype(np.float)
/tmp/ipykernel_45258/1522112106.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_test = x_test.astype(np.float)
/home/lkhagvadorj/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarn

In [17]:
#train , val
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train_one_hot, test_size=0.4)
print(x_train.shape) 
print(x_val.shape) 

(36000, 784)
(24000, 784)


In [ ]:
learning_rate = 0.01
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = x_train.shape[1]
n_samples = x_train.shape[0]
n_classes = 10

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

get_mini_batch_train = GetMiniBatch(x_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] 
    return layer_output
                             
logits = example_net(X)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: x_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: x_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/tmp/ipykernel_45258/1704943362.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
/tmp/ipykernel_45258/217344465.py:55: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional informat

Epoch 0, loss : 2.7374, val_loss : 1.3441, acc : 0.700, val_acc : 0.610
Epoch 1, loss : 4.8243, val_loss : 1.1141, acc : 0.900, val_acc : 0.645
Epoch 2, loss : 1.4630, val_loss : 0.9842, acc : 0.900, val_acc : 0.725
Epoch 3, loss : 1.2008, val_loss : 0.9319, acc : 0.900, val_acc : 0.768
Epoch 4, loss : 0.7674, val_loss : 0.6764, acc : 0.800, val_acc : 0.835
Epoch 5, loss : 0.4747, val_loss : 0.4681, acc : 0.800, val_acc : 0.886
Epoch 6, loss : 0.2550, val_loss : 0.4071, acc : 0.900, val_acc : 0.901
Epoch 7, loss : 0.4328, val_loss : 0.4041, acc : 0.900, val_acc : 0.903
Epoch 8, loss : 0.8776, val_loss : 0.4288, acc : 0.800, val_acc : 0.903
Epoch 9, loss : 0.6516, val_loss : 0.3854, acc : 0.800, val_acc : 0.916
Epoch 10, loss : 0.6349, val_loss : 0.4547, acc : 0.800, val_acc : 0.891
Epoch 11, loss : 0.6388, val_loss : 0.3800, acc : 0.800, val_acc : 0.923
Epoch 12, loss : 0.8099, val_loss : 0.3753, acc : 0.900, val_acc : 0.921
Epoch 13, loss : 0.5199, val_loss : 0.3887, acc : 0.900, val_